In [1]:
pip install opencv-python


     ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/39.5 MB 1.4 MB/s eta 0:00:29
     --------------------------------------- 0.0/39.5 MB 393.8 kB/s eta 0:01:41
     --------------------------------------- 0.1/39.5 MB 655.4 kB/s eta 0:01:01
     --------------------------------------- 0.1/39.5 MB 778.5 kB/s eta 0:00:51
     --------------------------------------- 0.2/39.5 MB 766.6 kB/s eta 0:00:52
     --------------------------------------- 0.2/39.5 MB 811.5 kB/s eta 0:00:49
     --------------------------------------- 0.2/39.5 MB 722.1 kB/s eta 0:00:55
     --------------------------------------- 0.3/39.5 MB 865.8 kB/s eta 0:00:46
     --------------------------------------- 0.3/39.5 MB 840.2 kB/s eta 0:00:47
     --------------------------------------- 0.4/39.5 MB 867.0 kB/s eta 0:00:46
     --------------------------------------- 0.4/39.5 MB 846.3 kB/s eta 0:00:47
     --------------------------------------- 0.5/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pyttsx3


     ---------------------------------------- 0.0/246.4 kB ? eta -:--:--
     ------ --------------------------------- 41.0/246.4 kB ? eta -:--:--
     ------------------------- ------------ 163.8/246.4 kB 2.0 MB/s eta 0:00:01
     -------------------------- ----------- 174.1/246.4 kB 1.8 MB/s eta 0:00:01
     -------------------------------------- 246.4/246.4 kB 1.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cv2
import time
import pyttsx3

# Load Haar cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize TTS engine
engine = pyttsx3.init()

# Start video capture
cap = cv2.VideoCapture(0)

# Blink logic variables
blink_count = 0
eyes_closed = False
last_blink_time = time.time()
triple_blink_times = []
long_eye_closure_start = None

# Message display control
message = ""
message_start_time = 0
message_spoken = False
current_command = ""

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)

        # Draw face rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Draw white ellipses on eyes
        for (ex, ey, ew, eh) in eyes:
            center = (x + ex + ew//2, y + ey + eh//2)
            axes = (ew//2, eh//2)
            angle = 0
            startAngle = 0
            endAngle = 360
            cv2.ellipse(frame, center, axes, angle, startAngle, endAngle, (255, 255, 255), 2)

        if len(eyes) >= 2:
            eyes_closed = False
            long_eye_closure_start = None
        else:
            # Blink detection
            if not eyes_closed and time.time() - last_blink_time > 0.5:
                blink_count += 1
                eyes_closed = True
                last_blink_time = time.time()

                triple_blink_times.append(last_blink_time)
                if len(triple_blink_times) > 3:
                    triple_blink_times.pop(0)

                # Triple blink detection
                if len(triple_blink_times) == 3 and triple_blink_times[-1] - triple_blink_times[0] < 2.5:
                    current_command = "I need help"
                    message = current_command
                    message_start_time = time.time()
                    message_spoken = False

                # Double blink detection
                elif len(triple_blink_times) >= 2 and triple_blink_times[-1] - triple_blink_times[-2] < 1.5:
                    current_command = "I want water"
                    message = current_command
                    message_start_time = time.time()
                    message_spoken = False

            # Long eye closure detection
            if long_eye_closure_start is None:
                long_eye_closure_start = time.time()
            elif time.time() - long_eye_closure_start > 2.0:
                current_command = "Call doctor"
                message = current_command
                message_start_time = time.time()
                message_spoken = False

    # Speak message once
    if message and not message_spoken:
        engine.say(message)
        engine.runAndWait()
        message_spoken = True

    # Remove message after 5 seconds
    if message and time.time() - message_start_time > 5:
        message = ""
        current_command = ""
        message_spoken = False

    # Show blink count
    cv2.putText(frame, f"Blinks: {blink_count}", (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the alert message
    if message:
        cv2.putText(frame, message, (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show video
    cv2.imshow('Blink Detection with Eye Ellipses', frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.
